In [36]:
pip install pytelegrambotapi

In [87]:
from google.colab import files

uploaded = files.upload()

Saving 0.txt to 0 (5).txt


In [37]:
pip install XlsxWriter

In [88]:
print(open('0 (5).txt').readlines())

['cho-to:15;\n', 'kak-to:16;\n', 'tito:846']


In [39]:
import pandas as pd
import random

In [116]:
de = pd.ExcelFile('bot.xlsx')
dl = de.parse('Лист1')
print(dl)

   Unnamed: 0  Unnamed: 0.1          ID    USERNAME  NUMBEROFBUYING    SUMM
0           0             0           1         grt            4654   45685
1           1             1           2          ft              56  468654
2           2             2           3       tyhyt               4     684
3           3             3   900506196  nekto_inoi               0       0
4           4             4   922328390     botskup               0       0
5           5             5   755458530    Виктория               0       0
6           6             6  1053539741      Кирилл               0       0


In [115]:
import telebot
import pandas as pd
import requests
import json
from datetime import datetime


bot = telebot.TeleBot('991410863:AAEDiQJtpqVRLoSTF-8IWTHqiYbrDe5kW_0')
kolvo_tovara = 3

mainmenu = telebot.types.ReplyKeyboardMarkup()
mainmenu.row(telebot.types.InlineKeyboardButton('/start', callback_data='start'), telebot.types.InlineKeyboardButton('Мой профиль', callback_data='profile'), telebot.types.InlineKeyboardButton('ПРАВИЛА', callback_data='RULES'))
mainmenu.row(telebot.types.InlineKeyboardButton('Наличие товара', callback_data='amount'))
mainmenu.row(telebot.types.InlineKeyboardButton('Открыть каталог товаров', callback_data='catalog'))

@bot.message_handler(commands=['start'])
def start_message(message):
    de = pd.ExcelFile('bot.xlsx')
    dl = de.parse('Лист1')
    if any(dl.ID == message.from_user.id):
      pass
    else:
      writer = pd.ExcelWriter('bot.xlsx', engine='xlsxwriter')
      dl.loc[len(dl)] = [len(dl),len(dl),str(message.from_user.id), str(message.from_user.first_name), 0, 0]
      dl = dl.drop('Unnamed: 0',axis=1)
      dl.to_excel(writer, 'Лист1')
      writer.save()
    an = str(message.from_user.username)
    an = an + ', категории обновлены!\nПрочтение FAQ - обязательно'
    bot.send_message(message.chat.id, an, reply_markup=mainmenu)
@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    if message.text == 'ПРАВИЛА':
      RULES = 'Какие-то правила'
      bot.send_message(message.chat.id, RULES, reply_markup=mainmenu)
    elif message.text == 'Мой профиль':
      de = pd.ExcelFile('bot.xlsx')
      dl = de.parse('Лист1')
      amnt = dl[dl['ID']==message.from_user.id]['NUMBEROFBUYING'].values[0]
      sum = dl[dl['ID']==message.from_user.id]['SUMM'].values[0]
      prof = 'Ваш ID: ' + str(message.from_user.id) + '\nВсего покупок: ' + str(amnt)+'\nСумма покупок: '+str(sum)+' руб.'
      bot.send_message(message.chat.id, prof, reply_markup=mainmenu)
    elif message.text == 'Наличие товара':
      de = pd.ExcelFile('Ships.xlsx')
      dl = de.parse('Лист1')
      sms = ''
      for lin in range(kolvo_tovara):
        sms = sms + str(dl.iloc[lin]['NAME'])+'//'+str(dl.iloc[lin]['COUNT'])+'шт.//'+str(dl.iloc[lin]['PRICE'])+' руб.\n'
      bot.send_message(message.chat.id, sms, reply_markup=mainmenu)
    elif message.text == 'Открыть каталог товаров':
      de = pd.ExcelFile('Ships.xlsx')
      dl = de.parse('Лист1')
      catalog = telebot.types.InlineKeyboardMarkup()
      for lin in range(kolvo_tovara):
        catalog.row(telebot.types.InlineKeyboardButton(str(dl.iloc[lin]['NAME']), callback_data=str(lin)))  
      bot.send_message(message.chat.id, 'Выберите категорию!', reply_markup=catalog)
@bot.callback_query_handler(func=lambda call: True)
def callback_inline(call):
  if call.data=='0' or call.data=='1' or call.data=='2':
    currenttov = ''
    key = telebot.types.InlineKeyboardMarkup()
    key.row(telebot.types.InlineKeyboardButton('Я согласен!', callback_data='GOOD'))
    key.row(telebot.types.InlineKeyboardButton('Не согласен!', callback_data='BAD'))
    bot.send_message(call.message.chat.id, 'Для продолжения покупки вы обязаны принять правила.\nПравила доступны в главном меню, кнопка "Правила"', reply_markup=key )
  elif call.data == 'BAD':
    currenttov = ''
    currenttov = ''
    de = pd.ExcelFile('Ships.xlsx')
    dl = de.parse('Лист1')
    catalog = telebot.types.InlineKeyboardMarkup()
    for lin in range(kolvo_tovara):
      catalog.row(telebot.types.InlineKeyboardButton(str(dl.iloc[lin]['NAME']), callback_data=str(lin)))  
    bot.send_message(call.message.chat.id, 'Выберите категорию!', reply_markup=catalog)
  elif call.data == 'GOOD':
    currenttov = ''
    de = pd.ExcelFile('Ships.xlsx')
    dl = de.parse('Лист1')
    catalog = telebot.types.InlineKeyboardMarkup()
    for lin in range(kolvo_tovara):
      catalog.row(telebot.types.InlineKeyboardButton(str(dl.iloc[lin]['NAME']), callback_data=str(-1*lin-1)))  
    bot.send_message(call.message.chat.id, 'Выберите категорию повторно, чтобы избежать несчатных случаев!', reply_markup=catalog)
  elif call.data == '-1':
    rd = random.randint(10000,99999)
    de = pd.ExcelFile('Ships.xlsx')
    dl = de.parse('Лист1')
    sms = 'ПОКУПКА ТОВАРА '+str(dl.iloc[0]['NAME'])+'\n Количество: 1 шт.\n ВРЕМЯ НА ОПЛАТУ ТОВАРА 10 минут\n Оплата через Qiwi\n Кошелёк +7.......\nСумма: '+str(dl.iloc[0]['PRICE'])+'\nКОММЕНТАРИЙ в QIWI: '+str(rd)+'\nПлатежи без комментариев не принимаются!!!\nВ комментарии указывать только цифры!\n\nПосле оплаты нажмите на кнопку ниже'
    key = telebot.types.InlineKeyboardMarkup()
    key.row(telebot.types.InlineKeyboardButton('Я оплатил!', callback_data='CHECK'+'0'+str(rd)))
    key.row(telebot.types.InlineKeyboardButton('Отменить заказ!', callback_data='BAD'))
    currenttov = '0'
    bot.send_message(call.message.chat.id, sms, reply_markup=key )
  elif call.data == '-2':
    rd = random.randint(10000,99999)
    de = pd.ExcelFile('Ships.xlsx')
    dl = de.parse('Лист1')
    sms = 'ПОКУПКА ТОВАРА '+str(dl.iloc[1]['NAME'])+'\n Количество: 1 шт.\n ВРЕМЯ НА ОПЛАТУ ТОВАРА 10 минут\n Оплата через Qiwi\n Кошелёк +7.......\nСумма: '+str(dl.iloc[1]['PRICE'])+'\nКОММЕНТАРИЙ в QIWI: '+str(rd)+'\nПлатежи без комментариев не принимаются!!!\nВ комментарии указывать только цифры!\n\nПосле оплаты нажмите на кнопку ниже'
    key = telebot.types.InlineKeyboardMarkup()
    key.row(telebot.types.InlineKeyboardButton('Я оплатил!', callback_data='CHECK'+'1'+str(rd)))
    key.row(telebot.types.InlineKeyboardButton('Отменить заказ!', callback_data='BAD'))
    currenttov = '1'
    bot.send_message(call.message.chat.id, sms, reply_markup=key )
  elif call.data == '-3':
    rd = random.randint(10000,99999)
    de = pd.ExcelFile('Ships.xlsx')
    dl = de.parse('Лист1')
    sms = 'ПОКУПКА ТОВАРА '+str(dl.iloc[2]['NAME'])+'\n Количество: 1 шт.\n ВРЕМЯ НА ОПЛАТУ ТОВАРА 10 минут\n Оплата через Qiwi\n Кошелёк +7.......\nСумма: '+str(dl.iloc[2]['PRICE'])+'\nКОММЕНТАРИЙ в QIWI: '+str(rd)+'\nПлатежи без комментариев не принимаются!!!\nВ комментарии указывать только цифры!\n\nПосле оплаты нажмите на кнопку ниже'
    key = telebot.types.InlineKeyboardMarkup()
    key.row(telebot.types.InlineKeyboardButton('Я оплатил!', callback_data='CHECK'+'2'+str(rd)))
    key.row(telebot.types.InlineKeyboardButton('Отменить заказ!', callback_data='BAD'))
    currenttov = '2'
    bot.send_message(call.message.chat.id, sms, reply_markup=key )
  elif call.data[0:5] == 'CHECK0':
    now = datetime.now()
    api_access_token = 'e08df1e7efea0c6f9f279bf57944bc1e' # токен
    my_login = '+380502752671' # номер QIWI Кошелька
    s = requests.Session()
    s.headers['authorization'] = 'Bearer ' + api_access_token  
    parameters = {'rows': 50}
    timing = False
    while ((datetime.now()-now).minute<1 and not(timing)):
      h = s.get('https://edge.qiwi.com/payment-history/v2/persons/' + my_login + '/payments', params = parameters)
      for line in h['data']:
        if line['comment']==call.data[6:10]:
          timing = True   
    if timing:
      de = pd.ExcelFile('Ships.xlsx')
      dl = de.parse('Лист1')
      if dl.iloc[0]['COUNT']>=1:
        f = open('0 (5).txt','r+')
        tovar = f.readline()
        lines = f.readlines()
        f.seek(0)
        for line in lines:
          if line!=tovar:
            f.write(line)
          f.truncate()
        f.close()
        bot.send_message(call.message.chat.id, tovar)
        writer = pd.ExcelWriter('Ships.xlsx', engine='xlsxwriter')
        dl.at[0, 'COUNT'] = dl.iloc[0]['COUNT']-1
        dl.to_excel(writer, 'Лист1')
        writer.save()
      else:
        bot.send_message(call.message.chat.id, 'Товар закончился')
    elif (datetime.now()-now).minute>1:
      bot.send_message(call.message.chat.id, 'Время истекло')
  elif call.data[0:5] == 'CHECK1':
    now = datetime.now()
    api_access_token = 'e08df1e7efea0c6f9f279bf57944bc1e' # токен
    my_login = '+380502752671' # номер QIWI Кошелька
    s = requests.Session()
    s.headers['authorization'] = 'Bearer ' + api_access_token  
    parameters = {'rows': 50}
    timing = False
    while ((datetime.now()-now).minute<1 and not(timing)):
      h = s.get('https://edge.qiwi.com/payment-history/v2/persons/' + my_login + '/payments', params = parameters)
      for line in h['data']:
        if line['comment']==call.data[6:10]:
          timing = True   
    if timing:
      de = pd.ExcelFile('Ships.xlsx')
      dl = de.parse('Лист1')
      if dl.iloc[1]['COUNT']>=1:
        f = open('1 (5).txt','r+')
        tovar = f.readline()
        lines = f.readlines()
        f.seek(0)
        for line in lines:
          if line!=tovar:
            f.write(line)
          f.truncate()
        f.close()
        bot.send_message(call.message.chat.id, tovar)
        writer = pd.ExcelWriter('Ships.xlsx', engine='xlsxwriter')
        dl.at[1, 'COUNT'] = dl.iloc[1]['COUNT']-1
        dl.to_excel(writer, 'Лист1')
        writer.save()
      else:
        bot.send_message(call.message.chat.id, 'Товар закончился')
    elif (datetime.now()-now).minute>1:
      bot.send_message(call.message.chat.id, 'Время истекло')
  elif call.data[0:5] == 'CHECK2':
    now = datetime.now()
    api_access_token = 'e08df1e7efea0c6f9f279bf57944bc1e' # токен
    my_login = '+380502752671' # номер QIWI Кошелька
    s = requests.Session()
    s.headers['authorization'] = 'Bearer ' + api_access_token  
    parameters = {'rows': 50}
    timing = False
    while ((datetime.now()-now).minute<1 and not(timing)):
      h = s.get('https://edge.qiwi.com/payment-history/v2/persons/' + my_login + '/payments', params = parameters)
      for line in h['data']:
        if line['comment']==call.data[6:10]:
          timing = True   
    if timing:
      de = pd.ExcelFile('Ships.xlsx')
      dl = de.parse('Лист1')
      if dl.iloc[2]['COUNT']>=1:
        f = open('2 (5).txt','r+')
        tovar = f.readline()
        lines = f.readlines()
        f.seek(0)
        for line in lines:
          if line!=tovar:
            f.write(line)
          f.truncate()
        f.close()
        bot.send_message(call.message.chat.id, tovar)
        writer = pd.ExcelWriter('Ships.xlsx', engine='xlsxwriter')
        dl.at[2, 'COUNT'] = dl.iloc[2]['COUNT']-1
        dl.to_excel(writer, 'Лист1')
        writer.save()
      else:
        bot.send_message(call.message.chat.id, 'Товар закончился')
    elif (datetime.now()-now).minute>1:
      bot.send_message(call.message.chat.id, 'Время истекло')
      
bot.polling()